In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        img = np.dstack((band_1, band_2, band_3))
        images.append(std_img(img))

    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 128)
    # 4x4x128
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            #zoom_range=0.05,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=50, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/50
Epoch 00001: val_loss improved from inf to 0.50011, saving model to best_m.h5
 - 3s - loss: 0.8306 - acc: 0.6681 - val_loss: 0.5001 - val_acc: 0.7506
Epoch 2/50
Epoch 00002: val_loss did not improve
 - 2s - loss: 0.4893 - acc: 0.7689 - val_loss: 0.7087 - val_acc: 0.6459
Epoch 3/50
Epoch 00003: val_loss did not improve
 - 2s - loss: 0.4482 - acc: 0.7922 - val_loss: 0.5233 - val_acc: 0.7656
Epoch 4/50
Epoch 00004: val_loss did not improve
 - 2s - loss: 0.4194 - acc: 0.8200 - val_loss: 0.5840 - val_acc: 0.7855
Epoch 5/50
Epoch 00005: val_loss improved from 0.50011 to 0.37980, saving model to best_m.h5
 - 2s - loss: 0.4045 - acc: 0.8314 - val_loss: 0.3798 - val_acc: 0.8529
Epoch 6/50
Epoch 00006: val_loss improved from 0.37980 to 0.26342, saving model to best_m.h5
 - 2s - loss: 0.3667 - acc: 0.8422 - val_loss: 0.2634 - val_acc: 0.8903
Epoch 7/50
Epoch 00007: val_loss did not improve
 - 2s - loss: 0.3475 - acc: 0.8491 - val_loss: 0.2704 - val_acc: 0.8703
Epoch 8/50
Epoch 00008: v

Epoch 14/50
Epoch 00014: val_loss did not improve
 - 2s - loss: 0.2528 - acc: 0.8925 - val_loss: 0.2791 - val_acc: 0.8579
Epoch 15/50
Epoch 00015: val_loss did not improve
 - 2s - loss: 0.2747 - acc: 0.8808 - val_loss: 0.2860 - val_acc: 0.8803
Epoch 16/50
Epoch 00016: val_loss did not improve
 - 2s - loss: 0.2435 - acc: 0.8898 - val_loss: 0.6366 - val_acc: 0.7731
Epoch 17/50
Epoch 00017: val_loss did not improve
 - 2s - loss: 0.2518 - acc: 0.8883 - val_loss: 0.3587 - val_acc: 0.8728
Epoch 18/50
Epoch 00018: val_loss did not improve
 - 2s - loss: 0.2682 - acc: 0.8831 - val_loss: 0.6780 - val_acc: 0.7781
Epoch 19/50
Epoch 00019: val_loss improved from 0.27152 to 0.26691, saving model to best_m.h5
 - 2s - loss: 0.2574 - acc: 0.8750 - val_loss: 0.2669 - val_acc: 0.8903
Epoch 20/50
Epoch 00020: val_loss did not improve
 - 2s - loss: 0.2472 - acc: 0.8883 - val_loss: 0.2767 - val_acc: 0.8853
Epoch 21/50
Epoch 00021: val_loss improved from 0.26691 to 0.25059, saving model to best_m.h5
 - 2s - 

Epoch 27/50
Epoch 00027: val_loss did not improve
 - 2s - loss: 0.2351 - acc: 0.9014 - val_loss: 0.2683 - val_acc: 0.8803
Epoch 28/50
Epoch 00028: val_loss did not improve
 - 2s - loss: 0.2506 - acc: 0.8889 - val_loss: 0.2293 - val_acc: 0.9077
Epoch 29/50
Epoch 00029: val_loss did not improve
 - 2s - loss: 0.2739 - acc: 0.8839 - val_loss: 0.2550 - val_acc: 0.9102
Epoch 30/50
Epoch 00030: val_loss did not improve
 - 2s - loss: 0.2236 - acc: 0.9033 - val_loss: 0.2259 - val_acc: 0.9102
Epoch 31/50
Epoch 00031: val_loss did not improve
 - 2s - loss: 0.2276 - acc: 0.9014 - val_loss: 0.2267 - val_acc: 0.9002
Epoch 32/50
Epoch 00032: val_loss did not improve
 - 2s - loss: 0.1969 - acc: 0.9225 - val_loss: 0.2104 - val_acc: 0.9127
Epoch 33/50
Epoch 00033: val_loss did not improve
 - 2s - loss: 0.2222 - acc: 0.9025 - val_loss: 0.2085 - val_acc: 0.9077
Epoch 34/50
Epoch 00034: val_loss improved from 0.20773 to 0.20193, saving model to best_m.h5
 - 2s - loss: 0.1980 - acc: 0.9181 - val_loss: 0.201

Epoch 00040: val_loss did not improve
 - 2s - loss: 0.1607 - acc: 0.9331 - val_loss: 0.2654 - val_acc: 0.8878
Epoch 41/50
Epoch 00041: val_loss did not improve
 - 2s - loss: 0.1730 - acc: 0.9208 - val_loss: 0.2654 - val_acc: 0.8903
Epoch 42/50
Epoch 00042: val_loss did not improve
 - 2s - loss: 0.1516 - acc: 0.9350 - val_loss: 0.2659 - val_acc: 0.8978
Epoch 43/50
Epoch 00043: val_loss did not improve
 - 2s - loss: 0.1586 - acc: 0.9308 - val_loss: 0.2696 - val_acc: 0.8928
Epoch 44/50
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.1668 - acc: 0.9291 - val_loss: 0.2729 - val_acc: 0.8978
Epoch 45/50
Epoch 00045: val_loss did not improve
 - 2s - loss: 0.1597 - acc: 0.9373 - val_loss: 0.3182 - val_acc: 0.9002
Epoch 46/50
Epoch 00046: val_loss did not improve
 - 2s - loss: 0.1557 - acc: 0.9283 - val_loss: 0.2999 - val_acc: 0.8978
Epoch 47/50
Epoch 00047: val_loss did not improve
 - 2s - loss: 0.1469 - acc: 0.9425 - val_loss: 0.2758 - val_acc: 0.8878
Epoch 48/50
Epoch 00048: val_loss di

In [6]:
import pickle
with open('../features/cnn_4_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_4_aug1_sub.csv', index=False)

0.216595971291
         id  is_iceberg
0  5941774d    0.005601
1  4023181e    0.893278
2  b20200e4    0.003259
3  e7f018bb    0.996785
4  4371c8c3    0.010084
